In [47]:
import numpy as np
import pandas as pd
import patsy
import matplotlib.pyplot as plt
from scipy import stats
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from statsmodels.stats import multicomp

In [67]:
# Paired t-Test and Wilcoxon signed rank sum test

# The daily energy intake from 11 healthy women is [5260., 5470., 5640., 6180.,
# 6390., 6515., 6805., 7515., 7515., 8230., 8770.] kJ.
# Is this value significantly different from the recommended value of 7725?

val = 7725
data = np.array([5260, 5470, 5640, 6180, 6390, 6515, 6805, 7515, 7515, 8230, 8770])

# 1-sample t-test
t, p_ttest = stats.ttest_1samp(data, val)
rank, p_wilcoxon = stats.wilcoxon(data-val)

print(f"p_ttest = {p_ttest:.3f}, and p_wilcoxon = {p_wilcoxon:.3f}, both of which are < 0.05, so there is a significant difference.")


p_ttest = 0.018, and p_wilcoxon = 0.024, both of which are < 0.05, so there is a significant difference.


In [68]:
# t-Test of independent samples

# In a clinic, 15 lazy patients weigh [76, 101, 66, 72, 88, 82, 79, 73, 76, 85, 75, 64,
# 76, 81, 86.] kg, and 15 sporty patients weigh [ 64, 65, 56, 62, 59, 76, 66, 82, 91,
# 57, 92, 80, 82, 67, 54] kg.
# Are the lazy patients significantly heavier?

data1 = np.array([76, 101, 66, 72, 88, 82, 79, 73, 76, 85, 75, 64, 76, 81, 86])
data2 = np.array([64, 65, 56, 62, 59, 76, 66, 82, 91, 57, 92, 80, 82, 67, 54])

t_stat, p_val = stats.ttest_ind(data1, data2)
p_val

print(f"p_val = {p_val:.3f}, which is < 0.05, so there is a significant difference.")



p_val = 0.045, which is < 0.05, so there is a significant difference.


In [69]:
# Normality test

# Are the two data sets normally distributed?

for dataset_name, dataset in [("data1", data1), ("data2", data2)]:
    (_, p) = stats.normaltest(dataset)
    if p > 0.05:
        a = " "
    else:
        a = " not "
    print(f'Data in {dataset_name} are{a}distributed normally, p = {p}')


Data in data1 are distributed normally, p = 0.19204806032435315
Data in data2 are distributed normally, p = 0.34678775457262334


c:\Users\rich\miniconda3\envs\pydata-book\lib\site-packages\scipy\stats\_stats_py.py:1736: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=15
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


In [70]:
# Mann–Whitney test

# Are the lazy patients still heavier, if you check with the Mann–Whitney test?

u, p_val = stats.mannwhitneyu(data1, data2)
print(f"With this test, p_val = {p_val:5.3f}, which is > 0.05, so there is no significant difference, i.e. lazy patients are not heavier.")

With this test, p_val = 0.077, which is > 0.05, so there is no significant difference, i.e. lazy patients are not heavier.


In [71]:
# The following example is taken from the really good, but somewhat advanced book
# by A.J. Dobson: “An Introduction to Generalized Linear Models”:

# The file Data/data_others/Table 6.6 Plant experiment.xls, which can also
# be found on https://github.com/thomas-haslwanter/statsintro/tree/master/Data/
# data_others, contains data from an experiment with plants in three different
# growing conditions. Read the data into Python. Hint: use the module xlrd.

In [72]:
# Perform an ANOVA

# Are the three groups different?

excel_file_path = "data/plant_experiment.xls"

# Read the Excel file into a pandas DataFrame
df = pd.read_excel(excel_file_path, header=2, usecols=['group', 'weight'])



In [74]:
# Let's fit a statistical "ordinary least square (ols)" model to the data.
# In the formula 'weight ~ C(group)', "weight" is a function of the categorical 
# value "group".

results = ols('weight ~ C(group)', df).fit()

In [75]:
# Peform the ANOVA:
anovaResults = anova_lm(results)
print(anovaResults)
    
# Because the probability of achieving this F-score under the null hypothesis
# of the samples is equal is < 0.05, we reject the null hypothesis.

            df    sum_sq   mean_sq         F   PR(>F)
C(group)   2.0   3.76634  1.883170  4.846088  0.01591
Residual  27.0  10.49209  0.388596       NaN      NaN


In [76]:
# Multiple Comparisons

# Using the Tukey test, which of the pairs are different?

mc = multicomp.MultiComparison(df['weight'], df['group'])

In [77]:
# Conduct Tukey Honest Significance Test (HSD) on the multicomparison,
# using default alpha=0.05

tukey = mc.tukeyhsd()
tukey.summary()

# TreatmentA-TreatmentB pair is the only one with p-value < 0.05,
# so we reject the null hypothesis and say Treatment A and Treatment B differ.

group1,group2,meandiff,p-adj,lower,upper,reject
Control,TreatmentA,-0.371,0.3909,-1.0622,0.3202,False
Control,TreatmentB,0.494,0.198,-0.1972,1.1852,False
TreatmentA,TreatmentB,0.865,0.012,0.1738,1.5562,True


In [79]:
# Kruskal–Wallis

# Would a nonparametric comparison lead to a different result?

group_a = df['weight'][df['group']=='TreatmentA']
group_b = df['weight'][df['group']=='TreatmentB']
group_c = df['weight'][df['group']=='Control']

# do Kruskal-Wallis test
h, p = stats.kruskal(group_c, group_a, group_b)
print(f'Result from Kruskal-Wallis test: p = {p}')

# p < 0.05, so we reject the null hypothesis that the median of all groups are equal.
# Note that rejecting the null hypothesis does not indicate which group(s) differ.

Result from Kruskal-Wallis test: p = 0.018423755731471966
